In [ ]:
import os
import zipfile
if os.path.exists('imagedb_btsd.zip'):
  if os.path.isdir('imagedb'):
    print('imagedb already exists')
  else:
    local_zip = 'imagedb_btsd.zip'
    zip_ref = zipfile.ZipFile(local_zip, 'r')
    # zip_ref.extractall('/content')
    zip_ref.extractall('./')
    zip_ref.close()
else:
  print('imagedb_btsd.zip not found')

In [ ]:
train_dir = 'imagedb'
test_dir = 'imagedb_test'

# train_dir = os.path.join(train_dir)
# test_dir = os.path.join(test_dir)
# validation_dir = os.path.join(base_dir, 'val')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
tf.keras.utils.set_random_seed(123)
train_datagen = ImageDataGenerator( rescale=1./255, validation_split=0.2,
                                   rotation_range=10,
                                   width_shift_range=0.1,
                                   zoom_range=0.1)
test_datagen = ImageDataGenerator( rescale=1./255, validation_split=0.2)

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=128,
                                                    class_mode='categorical',
                                                    color_mode='rgb',
                                                    target_size=(128,128),
                                                    subset='training')
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory(train_dir,
                                                         batch_size=128,
                                                         class_mode  = 'categorical',
                                                         color_mode='rgb',
                                                         target_size=(128,128),
                                                         subset='validation')

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (5, 5), strides=(1, 1), activation='relu', input_shape=(128, 128, 3)),
  tf.keras.layers.Conv2D(32, (3, 3), strides=(1, 1), activation='relu', input_shape=(128, 128, 3)),
  tf.keras.layers.Conv2D(64, (3, 3), strides=(1, 1), activation='relu', input_shape=(128, 128, 3)),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(34, activation='softmax')
])

model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
callbacks = []
save_best_callback = tf.keras.callbacks.ModelCheckpoint(f'best_weights.keras',
                                                        save_best_only=True,
                                                        verbose=1)
callbacks.append(save_best_callback)

early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, verbose=1)
callbacks.append(early_stop_callback)

history = model.fit(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=int(train_generator.samples/train_generator.batch_size),
                              epochs=50,
                              validation_steps=15,
                              verbose=1,
                              callbacks=callbacks)

In [ ]:
test_datagen  = ImageDataGenerator()
# test_dir = os.path.join(test_dir, 'test')
test_generator =  test_datagen.flow_from_directory(test_dir,
                                                   batch_size=128,
                                                   class_mode  = 'categorical',
                                                   color_mode='rgb',
                                                   target_size=(128,128))
loss, acc = model.evaluate(test_generator)
ff = open("Results.txt", "a")
ff.write(f"Accuracy of the model is: {round(acc*100,5)}%")
ff.write(f"\nLoss of the model is: {round(loss,5)}\n\n")
ff.close()

ff = open("Results.txt", "r")
print(ff.read())
ff.close()

In [ ]:
from posix import waitid_result
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.utils import load_img, img_to_array
import numpy as np
counter = 0
# for every class in train_dir
for s in os.listdir(train_dir):
  image = []
  for img in os.listdir(os.path.join(train_dir, s)):
    img_path = os.path.join(train_dir, s, img)
    # img = mpimg.imread(img_path)
    # image.append(img)
    counter += 1
    # plt.imshow(img, cmap='gray')
    # plt.show()
  # signs.append(image)
# print(counter)

# Just an example...
path = '/content/imagedb_test/00053/00001_00001.ppm'
img = load_img(path, target_size=(128, 128), interpolation='bilinear')
plt.imshow(img, cmap='gray')
plt.show()

x = img_to_array(img)
x = np.expand_dims(x, axis=0)
classes_pred = model.predict(x)
dir = os.listdir(test_dir)
dir.sort()
classes = [s for s in dir]
print(f'\n\n\n{classes}')
print(f'\n\n\nSoftmax Output: {classes_pred}')
print(f'\n\n\n{path} is a {classes[classes_pred.argmax()]}\n\n\n')